In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
raw_df = pd.read_csv("rawdata.csv",encoding='ISO-8859-1')
var_df = pd.read_csv("Varlist.csv")

In [3]:
print(raw_df.columns.to_list())

['STARTDATE', 'ENDDATE', 'PROGRESS', 'DURATION', 'FINISHED', 'RECORDEDDATE', 'DISTRIBUTIONCHANNEL', 'USERLANGUAGE', 'ARM', 'CONSENT', 'COMPREHENSION1_ARM1', 'COMPREHENSION1_ARM2', 'COMPREHENSION2_ARM1', 'COMPREHENSION2_ARM2', 'HR1', 'HR1_10SCALE', 'HR2_1', 'HR2_2', 'HR2_3', 'HR2_4', 'HR2_5', 'HR3', 'HR4', 'HR5', 'HR6', 'HR7_1', 'HR7_2', 'HR7_3', 'HR7_4', 'HR8', 'HR10', 'HR11', 'HR11_6_TEXT', 'HR12_1', 'HR12_2', 'HR12_3', 'HR12_4', 'HR12_5', 'HR13', 'HR14', 'HR14_10_TEXT', 'COVID_HR1', 'COVID_HR2', 'COVID_HR2_11_TEXT', 'COVID_HR3', 'STIMULUS1', 'STIMULUS2', 'STIMULUS3', 'STIMULUS4', 'STIMULUS5', 'STIMULUS5_8_TEXT', 'STIMULUS6', 'STIMULUS6_7_TEXT', 'POLICY1', 'POLICY2', 'POLICY3', 'POLICY4', 'POLICY5', 'POLICY5_13_TEXT', 'POLICY6', 'POLICY6_10_TEXT', 'PERSONAL1', 'PERSONAL2', 'PERSONAL3', 'PERSONAL4', 'PERSONAL5', 'PERSONAL6', 'PERSONAL6_13_TEXT', 'PERSONAL7', 'PERSONAL7_11_TEXT', 'WORRY_1', 'WORRY_2', 'WORRY_3', 'WORRY_4', 'WORRY_5', 'WORRY_6', 'WORRY_7', 'WORRY_8', 'MEDIA1', 'MEDIA2', 

### Figuring out what the types for each feature is

In [4]:
strings = raw_df[raw_df.select_dtypes(include=['object']).columns].columns.to_list()
print(strings)
floats = raw_df[raw_df.select_dtypes(include=['int64']).columns].columns.to_list()
integers = raw_df[raw_df.select_dtypes(include=['float64']).columns].columns.to_list()
nums = floats+integers

['STARTDATE', 'ENDDATE', 'RECORDEDDATE', 'DISTRIBUTIONCHANNEL', 'USERLANGUAGE', 'CONSENT', 'COMPREHENSION1_ARM1', 'COMPREHENSION1_ARM2', 'COMPREHENSION2_ARM1', 'COMPREHENSION2_ARM2', 'HR2_1', 'HR2_2', 'HR2_3', 'HR2_4', 'HR2_5', 'HR3', 'HR4', 'HR5', 'HR6', 'HR7_1', 'HR7_2', 'HR7_3', 'HR7_4', 'HR8', 'HR10', 'HR11', 'HR11_6_TEXT', 'HR12_1', 'HR12_2', 'HR12_3', 'HR12_4', 'HR12_5', 'HR13', 'HR14', 'HR14_10_TEXT', 'COVID_HR1', 'COVID_HR2', 'COVID_HR2_11_TEXT', 'COVID_HR3', 'STIMULUS1', 'STIMULUS2', 'STIMULUS3', 'STIMULUS4', 'STIMULUS5', 'STIMULUS5_8_TEXT', 'STIMULUS6', 'STIMULUS6_7_TEXT', 'POLICY1', 'POLICY2', 'POLICY3', 'POLICY4', 'POLICY5', 'POLICY5_13_TEXT', 'POLICY6', 'POLICY6_10_TEXT', 'PERSONAL1', 'PERSONAL2', 'PERSONAL3', 'PERSONAL4', 'PERSONAL5', 'PERSONAL6', 'PERSONAL6_13_TEXT', 'PERSONAL7', 'PERSONAL7_11_TEXT', 'WORRY_1', 'WORRY_2', 'WORRY_3', 'WORRY_4', 'WORRY_5', 'WORRY_6', 'WORRY_7', 'WORRY_8', 'MEDIA1', 'MEDIA2', 'MEDIA3', 'MEDIA3_11_TEXT', 'MEDIA4', 'MEDIA4_11_TEXT', 'MEDIA5',

cont: Duration, 
ratio: demographics3, duration, elderlycare1_1
ordinal: HR1_10SCALE
nominal: arm, hr1, 

In [5]:
cat_features = strings + ['HR1_10SCALE', 'ARM', 'HR1']
cont_features = ["DURATION", "DEMOGRAPHIC3", "ELDERLYCARE1_1"]

### Summary tables

In [6]:
import numpy as np
def cont_summary(feature_name, data):
    summary_feature_names = ['Feature', 'Desc.', 'Count', '% of Missing', 'Card.', 'Min.', 'Q1', 'Median', 
                            'Q3', 'Max.', 'Mean', 'Std. Dev.', 'Notes']
    # Your answer to Q1 goes here! Make sure to return a DataFrame with the features specified. 
    newDf = pd.DataFrame(columns=summary_feature_names)
    newDf.at[0,'Feature'] = feature_name
    newDf.at[0,'Card.'] = data[feature_name].nunique()
    newDf.at[0,'Count'] = data[feature_name].count()
    newDf.at[0,'% of Missing'] = (1-data[feature_name].count()/data[feature_name].size)*100
    newDf.at[0,'Min.'] = data[feature_name].min()
    newDf.at[0,'Q1'] = data[feature_name].quantile(0.25)
    newDf.at[0,'Mean'] = data[feature_name].mean()
    newDf.at[0,'Median'] = data[feature_name].quantile(0.50)
    newDf.at[0,'Q3'] = data[feature_name].quantile(.75)
    newDf.at[0,'Max.'] = data[feature_name].max()
    newDf.at[0,'Std. Dev.'] = data[feature_name].std()
    newDf.at[0,'Notes'] = ""
    newDf.at[0,'Desc.'] = ""
    IQR = data[feature_name].quantile(.75) - data[feature_name].quantile(0.25)
    Q1 = data[feature_name].quantile(0.25)
    Q3 = data[feature_name].quantile(.75)
    return newDf

def cat_summary(feature_name,data):
    summary_feature_names = ['Feature', 'Desc.', 'Count', '% of Missing', 'Card.', 'Mode', 'Mode Freq.', 'Mode %', 
                            '2nd Mode', '2nd Mode Freq.', '2nd Mode Perc', 'Notes']
    if feature_name == 'HR1_10SCALE':
        data[feature_name] = data[feature_name].astype("string") 
    newDf = pd.DataFrame(columns=summary_feature_names)
    newDf.at[0,'Feature'] = feature_name
    newDf.at[0,'Card.'] = data[feature_name].nunique()
    newDf.at[0,'Count'] = data[feature_name].count()
#     if '?' in data[feature_name].tolist():
#         #np.count(np.array(data[feature_name].tolist()) == '?')
#         newDf.at[0,'% of Missing'] = (np.count_nonzero(np.array(data[feature_name].tolist()) == '?')/data[feature_name].size)*100
#     else:
#         newDf.at[0,'% of Missing'] = (1-data[feature_name].count()/data[feature_name].size)*100
    newDf.at[0,'% of Missing'] = ((data[feature_name].isna().count() + data[feature_name].isnull().count())/data[feature_name].size)*100
    newDf.at[0,'Mode'] = data[feature_name].value_counts().index[0]
    newDf.at[0,'Mode Freq.'] = data[feature_name].value_counts()[0]
    newDf.at[0,'Mode %'] = data[feature_name].value_counts()[0]/data[feature_name].count()*100
    if data[feature_name].value_counts()[0]/data[feature_name].count()*100 != 100:
        newDf.at[0,'2nd Mode'] = data[feature_name].value_counts().index[1]
        newDf.at[0,'2nd Mode Freq.'] = data[feature_name].value_counts()[1]
        newDf.at[0,'2nd Mode Perc'] = data[feature_name].value_counts()[1]/data[feature_name].count()*100
    newDf.at[0,'Notes'] = ""
    newDf.at[0,'Desc.'] = ""
    return newDf

In [7]:
summary_feature_names = ['Feature', 'Desc.', 'Count', '% of Missing', 'Card.', 'Min.', 'Q1', 'Median','Q3', 'Max.', 'Mean', 'Std. Dev.', 'Notes']
quality_cont = pd.DataFrame(columns=summary_feature_names)
for i in cont_features:
    tmp = cont_summary(i,raw_df)
    quality_cont = quality_cont.append(tmp.loc[0], ignore_index=True)
print(quality_cont)
#quality_cont.to_csv("cont_summary.csv")

summary_feature_names = ['Feature', 'Desc.', 'Count', '% of Missing', 'Card.', 'Mode', 'Mode Freq.', 'Mode %','2nd Mode', '2nd Mode Freq.', '2nd Mode Perc', 'Notes']
quality_cat = pd.DataFrame(columns=summary_feature_names)
for i in cat_features:
    print(i)
    if i!= "HR1":
        tmp = cat_summary(i,raw_df)
        quality_cat = quality_cat.append(tmp.loc[0], ignore_index=True)
print(quality_cat)

#quality_cat.to_csv("cat_summary.csv")

          Feature Desc. Count  % of Missing Card.  Min.      Q1  Median  \
0        DURATION        1211      0.000000   983   232   717.5  1168.0   
1    DEMOGRAPHIC3        1211      0.000000    80  1924  1971.0  1982.0   
2  ELDERLYCARE1_1         331     72.667217    30   0.0     1.0     1.0   

       Q3          Max.          Mean     Std. Dev. Notes  
0  1881.5         45050  1.698932e+03  2.354988e+03        
1  1990.0          2010  1.979171e+03  1.584193e+01        
2     2.0  9867436378.0  2.981144e+07  5.423633e+08        
STARTDATE
ENDDATE
RECORDEDDATE
DISTRIBUTIONCHANNEL
USERLANGUAGE
CONSENT
COMPREHENSION1_ARM1
COMPREHENSION1_ARM2
COMPREHENSION2_ARM1
COMPREHENSION2_ARM2
HR2_1
HR2_2
HR2_3
HR2_4
HR2_5
HR3
HR4
HR5
HR6
HR7_1
HR7_2
HR7_3
HR7_4
HR8
HR10
HR11
HR11_6_TEXT
HR12_1
HR12_2
HR12_3
HR12_4
HR12_5
HR13
HR14
HR14_10_TEXT
COVID_HR1
COVID_HR2
COVID_HR2_11_TEXT
COVID_HR3
STIMULUS1
STIMULUS2
STIMULUS3
STIMULUS4
STIMULUS5
STIMULUS5_8_TEXT
STIMULUS6
STIMULUS6_7_TEXT
POLICY1
POL

Our main features are these. Of course they are broken into multiple groups for each but this is mainly what we are working with.
'STARTDATE', 'ENDDATE', 'PROGRESS', 'DURATION', 'FINISHED', 'RECORDEDDATE', 'DISTRIBUTIONCHANNEL', 'USERLANGUAGE', 'ARM', 'CONSENT', 'COMPREHENSION1_ARM1', 'HR1',  'COVID_HR1', 'STIMULUS1', 'POLICY1', 'PERSONAL1', 'WORRY_1', 'MEDIA1',  'MH1', 'CHILDCARE1','ELDERLYCARE1', 'IDEOLOGY1', 'CULTURE1_1', 'RUMORS1','DEMOGRAPHIC1'

In [8]:
bad_features = []
for index, row in quality_cat.iterrows():
    if quality_cat['Card.'][index] > 9:
        bad_features.append(quality_cat['Feature'][index])

for index, row in quality_cat.iterrows():
    if quality_cat['% of Missing'][index] != 0:
        print("test")
        bad_features.append(quality_cat['Feature'][index])
bad_features = list(set(bad_features))
print(bad_features)

test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
test
['HR2_5', 'STIMULUS2', 'STIMULUS5', 'MEDIA6_2', 'MEDIA2', 'IDEOLOGY4', 'PERSONAL1', 'CHILDCARE2', 'STIMULUS1', 'WORRY_2', 'HR13', 'CULTURE2_3', 'PERSONAL5', 'HR2_3', 'MEDIA3', 'COVID_HR3', 'HR10', 'EN

### Setting up the data scales

This is all just me creating an encoded df for nominal data. We still need to do ordinal

In [9]:
scales_df = pd.read_csv("DataScales.csv")

In [10]:
scales = scales_df[["FieldName","Data Scale"]]
nominal = scales.loc[scales["Data Scale"] == "nominal", "FieldName"].to_list()
tmp = []
for i in nominal:
    if i[-4:] != "TEXT":
        tmp.append(i)
nominal = tmp
#for now we are only using values with cardinality less than 9 and don't have missing values

ordinal = scales.loc[scales["Data Scale"] == "ordinal", "FieldName"].to_list()
tmp = []
for i in ordinal:
    if i[-4:] != "TEXT":
        tmp.append(i)
ordinal = tmp
ordinal

['HR1_10SCALE',
 'HR2_1',
 'HR2_2',
 'HR2_3',
 'HR2_4',
 'HR2_5',
 'HR3',
 'HR4',
 'HR5',
 'HR6',
 'HR7_1',
 'HR7_2',
 'HR7_3',
 'HR7_4',
 'HR8',
 'HR12_1',
 'HR12_2',
 'HR12_3',
 'HR12_4',
 'HR12_5',
 'STIMULUS3',
 'STIMULUS4',
 'POLICY1',
 'POLICY2',
 'POLICY3',
 'PERSONAL2',
 'PERSONAL3',
 'PERSONAL4',
 'WORRY_1',
 'WORRY_2',
 'WORRY_3',
 'WORRY_4',
 'WORRY_5',
 'WORRY_6',
 'WORRY_7',
 'WORRY_8',
 'MEDIA1',
 'MEDIA6_1',
 'MEDIA6_2',
 'MEDIA6_3',
 'MEDIA6_4',
 'MEDIA6_5',
 'MEDIA6_6',
 'MEDIA6_7',
 'MEDIA7',
 'MEDIA10',
 'MEDIA11',
 'MEDIA12',
 'MH1',
 'MH2',
 'IDEOLOGY5',
 'IDEOLOGY6',
 'CULTURE1_1',
 'CULTURE1_2',
 'CULTURE1_3',
 'CULTURE1_4',
 'CULTURE1_5',
 'CULTURE1_6',
 'CULTURE2_1',
 'CULTURE2_2',
 'CULTURE2_3',
 'CULTURE2_4',
 'CULTURE2_5',
 'CULTURE2_6',
 'CULTURE2_7',
 'DEMOGRAPHIC9',
 'DEMOGRAPHIC11',
 'DEMOGRAPHIC13']

In [11]:
tmp = nominal
for i in tmp:
    if i in bad_features:
        nominal.remove(i)

In [12]:
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
encoding_df = pd.DataFrame(columns=["Name","Values","Encoded Values"])
nn_df = pd.DataFrame(columns = nominal)


names = []
values = []
evalues = []
for i in nominal:
    tmp = []
    encoder = OneHotEncoder(sparse=False)
    
    for j in raw_df[i].value_counts().index.to_list():
        tmp.append([j])
    if tmp != []:
        data = tmp
        onehot = encoder.fit_transform(data)
        names.append(i)
        values.append(data)
        evalues.append(onehot)
        tmp = []
        for k in raw_df[i].to_list():
            if [k] in data:
                tmp.append(onehot[data.index([k])])
            else:
                tmp.append([k])
        nn_df[i] = tmp
    
encoding_df["Name"] = names
encoding_df["Values"] = values
encoding_df["Encoded Values"] = evalues

In [14]:
print(encoding_df)
nn_df

                   Name                                             Values  \
0               CONSENT                                [[Yes, I consent.]]   
1   COMPREHENSION2_ARM1  [[Hotel felt pressure from Airbnb and shut-dow...   
2   COMPREHENSION2_ARM2  [[He had employer-sponsored health coverage th...   
3                  HR11  [[Creating a universal Medicare-for-All system...   
4                  HR14  [[Private health insurance plan sponsored thro...   
5             COVID_HR2  [[C.\tA combination of both.], [B.\tHealth ins...   
6             STIMULUS1  [[Nothing changed (i.e., was retired; already ...   
7             STIMULUS5  [[Yes, check from the government], [No], [Yes,...   
8               POLICY4  [[People should not put economic concerns abov...   
9               POLICY6  [[Encourage people to engage in voluntary soci...   
10            PERSONAL5  [[Wash hands more often], [Wash hands more oft...   
11            PERSONAL7  [[None of the above], [Reduced income],

,CONSENT,COMPREHENSION2_ARM1,COMPREHENSION2_ARM2,HR1,HR11,HR14,COVID_HR2,STIMULUS1,STIMULUS5,POLICY4,...,CHILDCARE3_2,IDEOLOGY1,IDEOLOGY3,IDEOLOGY7,RUMORS1,DEMOGRAPHIC1,DEMOGRAPHIC3,DEMOGRAPHIC5,DEMOGRAPHIC7,DEMOGRAPHIC10
0,[1.0],"[1.0, 0.0]","[1.0, 0.0]",NaN,"[0.0, 1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0]",...,[nan],"[0.0, 0.0, 1.0, 0.0]","[0.0, 1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0]","[0.0, 1.0, 0.0]"
1,[1.0],[nan],[nan],NaN,"[1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0]",...,[nan],"[0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0]","[1.0, 0.0, 0.0]"
2,[1.0],[nan],[nan],NaN,"[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0]",...,[nan],"[1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0]","[1.0, 0.0, 0.0]"
3,[1.0],[nan],[nan],NaN,"[0.0, 0.0, 1.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[nan],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0]",...,[nan],"[0.0, 0.0, 1.0, 0.0]","[1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0]","[1.0, 0.0, 0.0]"
4,[1.0],"[1.0, 0.0]","[0.0, 1.0]",NaN,"[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0]",...,[nan],"[0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0]","[1.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,[1.0],[nan],[nan],NaN,"[0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",[nan],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0]",...,[nan],"[1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 1.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0]","[1.0, 0.0, 0.0]"
1207,[1.0],[nan],[nan],NaN,"[0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",[nan],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0]",...,[nan],"[0.0, 0.0, 1.0, 0.0]","[0.0, 1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0

In [15]:
encoding_df.to_csv("Encoding.csv")

### Messing around with some learning

In [ ]:
#drop STARTDATE ENDDATE FINISHED	RECORDEDDATE	DISTRIBUTIONCHANNEL

In [28]:
from sklearn.model_selection import train_test_split
test = ["STARTDATE","ENDDATE","FINISHED","RECORDEDDATE","DISTRIBUTIONCHANNEL", "PROGRESS"]
cols = []
for i in raw_df.columns.to_list():
    if i not in test and i[:2] != "HR":
        cols.append(i)
X = raw_df[cols]
print(X)
y = raw_df["HR12_4"] #obama care
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = .75, test_size = .25, random_state = 72)

      DURATION USERLANGUAGE  ARM          CONSENT  \
0          370           EN    1  Yes, I consent.   
1          447           EN    0  Yes, I consent.   
2          478           EN    2  Yes, I consent.   
3          605           EN    2  Yes, I consent.   
4          275           EN    1  Yes, I consent.   
...        ...          ...  ...              ...   
1206       448           EN    0  Yes, I consent.   
1207       300           EN    0  Yes, I consent.   
1208       680           EN    0  Yes, I consent.   
1209       585           EN    0  Yes, I consent.   
1210      1426           EN    1  Yes, I consent.   

                                COMPREHENSION1_ARM1  \
0                                               NaN   
1                                               NaN   
2     Hotel felt pressure from Airbnb and shut-down   
3          Coronavirus caused his hotel to shutdown   
4                                               NaN   
...                              

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [29]:
clf = DecisionTreeClassifier(max_depth = 2)
print(y_train)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print("Error rate:", 1 - metrics.accuracy_score(y_test,y_pred))
cfm = pd.DataFrame(metrics.confusion_matrix(y_test,y_pred))
print(cfm)

518     Mostly Positive
649     Mostly Positive
280     Mostly Negative
426          No Opinion
296          No Opinion
             ...       
1031    Mostly Negative
837     Mostly Negative
1098    Mostly Negative
1070    Mostly Positive
472     Mostly Positive
Name: HR12_4, Length: 908, dtype: object


ValueError: could not convert string to float: 'EN'

In [ ]:
test